In [ ]:
import base64
import os
with open("../secrets.json") as secrets:
    secrets_dict = eval(secrets.read())
    open_api_key = base64.b64decode(secrets_dict["openai_api_key"]).decode('ascii')
    os.environ["OPENAI_API_KEY"] = open_api_key
    if "organization_id" in secrets_dict.keys():
        openai_organization = base64.b64decode(secrets_dict["organization_id"]).decode('ascii')
        os.environ["OPENAI_ORGANIZATION"] = openai_organization
del open_api_key, openai_organization, secrets_dict

# Tree Sitter

## Execute from module

In [ ]:
# os.chdir("../examples/java_repo_s")
# os.chdir("../examples/ml_repo_s")
os.chdir("../examples/js_repo_s")

In [ ]:
# Run it just once
# from tree_sitter import Language
# Language.build_library(
#   # Store the library in the `build` directory
#   'build/tree-sitter-grammars.so',

#   # Include one or more languages
#   [
#     '/Users/itortouch/GiTor/tree-sitter-python',
#     '/Users/itortouch/GiTor/tree-sitter-javascript',
#     '/Users/itortouch/GiTor/tree-sitter-java'
#   ]
# )

In [ ]:
from codeas.ts_assistant import ts_Assistant

In [ ]:
# TODO: Pydantinc does not allow to pass language as input by any means. By now, change language at the source code directly.
ts_ca = ts_Assistant()

In [ ]:
# ts_ca.execute_prompt(instructions="Create java docs.", modules=["StringUtil"])
# ts_ca.execute_prompt(instructions="Create docstrings.", modules=["ml_pipeline"])
ts_ca.execute_prompt(instructions="Create docstrings.", modules=["ClassA"])

In [ ]:
ts_ca.apply_changes()

In [ ]:
ts_ca.revert_changes()

In [ ]:
ts_ca.reject_changes()